In [114]:
def ListofExtensionAndName(directory,extension):
     
        if len(directory) != None:
            import os
            FilesList = []
            FileName=[]
            for root, subdirectory, files in os.walk(directory):
                for file in files:
                    if file.endswith(extension):
                        FilesList.append(os.path.join(root,file))
                        base=os.path.basename(file)
                        FileName.append(os.path.splitext(base)[0])

            return sorted(FilesList),sorted(FileName)
        else:
            print("no"+ extension +"file for this directory")

In [113]:
import config
username=config.username
password=config.password

In [111]:
import time
import sys

In [88]:
from bs4 import BeautifulSoup
import os
import re
import requests
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
import requests
from osgeo import gdal,osr
import numpy
# If you are running Firefox on a system with no display, make sure you use headless mode.
# We use Selenium to export information from map
options = Options()
options.headless = True
driver = webdriver.Firefox(options=options)
url='https://e4ftl01.cr.usgs.gov/MOLA/MYD17A3HGF.006/'
driver.get(url)

In [2]:
modis_page=BeautifulSoup(driver.page_source,'html.parser')

In [7]:
year_list=modis_page.find_all('a')[8:]
year_list[0:2]

[<a href="2002.07.04/">2002.07.04/</a>, <a href="2003.01.01/">2003.01.01/</a>]

In [12]:
main_folder='../geoserver_data/2020_Modis_NPP/'

In [19]:
for year in year_list:
    folder_path=main_folder+str(year['href'])
    os.mkdir(folder_path)

In [78]:
import http.cookiejar as CookieJar
from urllib.parse import urlencode 
import urllib.request as urllib2

In [ ]:
for year in year_list:
    url='https://e4ftl01.cr.usgs.gov/MOLA/MYD17A3HGF.006/'
    url_year=url+str(year['href'])
    year_info=int(str(year['href']).split('.')[0])
    if year_info<2018:
        print(year_info)
        continue
    page=requests.get(url_year)
    year_page=BeautifulSoup(page.content,'html.parser')
    links = year_page.findAll(href=re.compile("\.hdf$"))
    download_path=main_folder+str(year['href'])
    
    for l in links:
        #raw_url='https://e4ftl01.cr.usgs.gov/MOLA/MYD17A3HGF.006/2002.07.04/'
        url_img=url_year+l['href']       
        
        username = username
        password = password
        password_manager = urllib2.HTTPPasswordMgrWithDefaultRealm()
        password_manager.add_password(None, "https://urs.earthdata.nasa.gov", username, password)
        cookie_jar = CookieJar.CookieJar()
        opener = urllib2.build_opener(urllib2.HTTPBasicAuthHandler(password_manager),
                                      #urllib2.HTTPHandler(debuglevel=1),    # Uncomment these two lines to see
                                      #urllib2.HTTPSHandler(debuglevel=1),   # details of the requests/responses
                                      urllib2.HTTPCookieProcessor(cookie_jar))
        urllib2.install_opener(opener)
        request = urllib2.Request(url_img)
        response = urllib2.urlopen(request)
        image_path=download_path+l['href'] 
        image_bool=False
        while image_bool is False:
            try:
                with open(image_path, 'wb') as f:
                    f.write(response.read())
                    image_bool=True
            except ConnectionRefusedError:
                time.sleep(10)
                image_bool=False
                pass
            except NewConnectionError:
                time.sleep(10)
                image_bool=False
                pass
            except MaxRetryError:
                time.sleep(10)
                image_bool=False
                pass
            except ConnectionError:
                time.sleep(10)
                image_bool=False
                pass
            except:
                time.sleep(10)
                print ("Unexpected error:", sys.exc_info()[0])
                image_bool=False
                pass
        
    modis_files=ListofExtensionAndName(download_path,'hdf')
    os.mkdir(download_path+'tif')
    
    for index,img in enumerate(modis_files[0]):
        img_path='HDF4_EOS:EOS_GRID:'+'"{}"'.format(img)+':MOD_Grid_MOD17A3H:Npp_500m'
        hdf_ds = gdal.Open(img_path, gdal.GA_ReadOnly)
        array_img=hdf_ds.ReadAsArray()
        np_where_img = numpy.where((array_img>32760),-32768,(array_img))
        check=numpy.all(np_where_img==-32768)
        if check:
            #print(img)
            continue    
        s_srs = hdf_ds.GetProjectionRef()
        osng = osr.SpatialReference ()
        osng.SetFromUserInput ( s_srs )
        geo_t = hdf_ds.GetGeoTransform ()
        x_size = hdf_ds.RasterXSize # Raster xsize
        y_size = hdf_ds.RasterYSize # Raster ysize
        #create gdal memory file to convert image to EPSG:4326
        mem_drv = gdal.GetDriverByName( 'MEM' )
        dest = mem_drv.Create('', x_size,y_size, 1, gdal.GDT_Int16)
        dest.SetGeoTransform( geo_t )
        dest.SetProjection ( osng.ExportToWkt() )
        dest.GetRasterBand(1).SetNoDataValue(-32768)
        dest.GetRasterBand(1).WriteArray(np_where_img)
        output_folder=os.path.dirname(img)
        output_path=output_folder+'/tif/'+modis_files[1][index]+'.tif'
        gdal.Warp(output_path, dest, format = 'GTiff',srcNodata=-32768,dstNodata=-32768)
        dst_ds=None
        mem_drv=None
        
        
    tif_list=ListofExtensionAndName(download_path+'tif','.tif')
    txt_name=year['href'][0:-1]+'.txt'
    # write all image' path into .txt file
    with open(txt_name, 'w') as f:
        for item in tif_list[0]:
            f.write("%s\n" % item)
    
    mosaic_tif_name='../geoserver_data/2020_Modis_NPP/NPP_Mosaic/'+'NPP_'+year['href'][0:-1]+'.tif'
    print(mosaic_tif_name)
    # use gdal_merge with .txt file that include images list
    command = "gdal_merge.py -n -32768 -a_nodata -32768 -ot Int16 -of GTiff -o %s --optfile %s" % (mosaic_tif_name, txt_name)

    # Run the command. os.system() returns value zero if the command was executed succesfully
    os.system(command)

       

In [115]:
#get the list of mosaic image
mosaic_list=ListofExtensionAndName('../geoserver_data/2020_Modis_NPP/NPP_Mosaic','tif')

#### Convert npp mosaic images to epsg:4326

In [123]:
output_folder='../geoserver_data/2020_Modis_NPP/NPP_Mosaic_4326/'
for img in mosaic_list[0]:
    img_name=os.path.basename(img)
    img_path=output_folder+img_name
    gdal.Warp(img_path, img, format = 'GTiff',srcSRS='+proj=sinu +R=6371007.181 +nadgrids=@null +wktext' ,dstSRS = 'EPSG:4326',srcNodata=-32768,dstNodata=-32768)
    
    